In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/CAPSTONE_DATASETS'

ALL_DOWN_SYNDROME_IMAGES  Deletion_syndrome  TEXTUAL_DATA
Autism_Images		  GMDB		     TRAIN1


In [ ]:
import os
import cv2

# Set the paths for your dataset and output
dataset_path = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/healthy"
output_path = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/HEALTHY_GRAY_IMAGES_EXTENDED"
os.makedirs(output_path, exist_ok=True)

# Function to convert image to grayscale
def convert_to_grayscale(image_path, output_path, filename):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Save the grayscale image to the output directory
    output_image_path = os.path.join(output_path, filename)
    cv2.imwrite(output_image_path, gray_image)

# Process all images in the dataset
for filename in os.listdir(dataset_path):
    if filename.endswith((".jpg", ".png")):
        image_path = os.path.join(dataset_path, filename)
        convert_to_grayscale(image_path, output_path, filename)

print("Grayscale conversion complete. Check:", output_path)


Grayscale conversion complete. Check: /content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/HEALTHY_GRAY_IMAGES_EXTENDED


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Input: Grayscale images (output of previous script)
input_path = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/HEALTHY_GRAY_IMAGES_EXTENDED"
augmented_path = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/HEALTHY_GRAY_IMAGES_AUGMENTED"
os.makedirs(augmented_path, exist_ok=True)

def augment_and_save(image, filename, count):
    base_name = os.path.splitext(filename)[0]

    # Save original grayscale image
    cv2.imwrite(os.path.join(augmented_path, f"{base_name}_orig.jpg"), image)

    # Horizontal Flip
    flipped = cv2.flip(image, 1)
    cv2.imwrite(os.path.join(augmented_path, f"{base_name}_flip.jpg"), flipped)

    # Rotation
    for angle in [15, -15]:
        (h, w) = image.shape
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_REFLECT)
        cv2.imwrite(os.path.join(augmented_path, f"{base_name}_rot{angle}.jpg"), rotated)

    # Zoom
    for zoom_factor in [1.1, 1.2]:
        zoomed = cv2.resize(image, None, fx=zoom_factor, fy=zoom_factor)
        zh, zw = zoomed.shape
        zh_crop = (zh - h) // 2
        zw_crop = (zw - w) // 2
        cropped_zoom = zoomed[zh_crop:zh_crop+h, zw_crop:zw_crop+w]
        cv2.imwrite(os.path.join(augmented_path, f"{base_name}_zoom{int(zoom_factor*10)}.jpg"), cropped_zoom)

    # Brightness adjustment
    for factor in [0.8, 1.2]:  # darker and brighter
        bright = np.clip(image * factor, 0, 255).astype(np.uint8)
        cv2.imwrite(os.path.join(augmented_path, f"{base_name}_bright{int(factor*10)}.jpg"), bright)

# Process all grayscale images
for count, filename in enumerate(tqdm(os.listdir(input_path))):
    if filename.endswith((".jpg", ".png")):
        img_path = os.path.join(input_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        augment_and_save(img, filename, count)

print("Augmentation complete. Check:", augmented_path)


100%|██████████| 842/842 [01:08<00:00, 12.26it/s]

Augmentation complete. Check: /content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/HEALTHY_GRAY_IMAGES_AUGMENTED


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define custom Dataset class to load images
class GrayscaleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("L")  # Convert to grayscale
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Set image transformations (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing to match ResNet input size
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalizing grayscale image
])

# Load the dataset
dataset_path = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/HEALTHY_GRAY_IMAGES_AUGMENTED"
labels = []  # List to hold the labels
image_paths = []  # List to hold the image paths

# Assuming the dataset is flat (not in subfolders)
for filename in os.listdir(dataset_path):
    if filename.endswith((".jpg", ".png")):
        image_paths.append(os.path.join(dataset_path, filename))
        labels.append(0)  # For now, assign label 0 (or change if you have multiple classes)

# Check if the dataset has been populated
print(f"Total image paths found: {len(image_paths)}")
print(f"Total labels found: {len(labels)}")

# Split the dataset into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = GrayscaleDataset(train_paths, train_labels, transform=transform)
val_dataset = GrayscaleDataset(val_paths, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load a pre-trained ResNet model
resnet = models.resnet18(pretrained=True)

# Modify the first convolution layer to accept single-channel (grayscale) images
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Modify the final fully connected layer to match the number of classes
num_classes = len(set(labels))  # Number of unique classes
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

# Move model to the device (GPU if available)
resnet.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Training loop
num_epochs = 10

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # Backpropagation
            optimizer.step()  # Optimizer step

            # Track training metrics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

        # Validation step
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        print(f"Validation Accuracy: {val_accuracy:.2f}%")

        # Save the best model based on validation accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), "best_resnet_model.pth")
            print("Model saved!")

# Train the model
train_model(resnet, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs)


Total image paths found: 6736
Total labels found: 6736


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Mish Activation Function
class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(nn.functional.softplus(x))

# Define custom Dataset class to load images
class GrayscaleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("L")  # Grayscale
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load dataset
dataset_path = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/HEALTHY_GRAY_IMAGES_EXTENDED"
labels = []
image_paths = []

for filename in os.listdir(dataset_path):
    if filename.endswith((".jpg", ".png")):
        image_paths.append(os.path.join(dataset_path, filename))
        labels.append(0)  # Adjust if using multiple classes

print(f"Total image paths found: {len(image_paths)}")

# Train-test split
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42)

# Dataset and dataloaders
train_dataset = GrayscaleDataset(train_paths, train_labels, transform=transform)
val_dataset = GrayscaleDataset(val_paths, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet-18
resnet = models.resnet18(pretrained=True)
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

num_classes = len(set(labels))  # Adjust for multiple classes

# Replace classifier with Mish-based head
resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 128),
    Mish(),
    nn.Linear(128, num_classes)
)

resnet.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        print(f"Validation Accuracy: {val_accuracy:.2f}%")

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), "best_resnet_mish_model.pth")
            print("Model saved!")

# Run training
train_model(resnet, train_loader, val_loader, criterion, optimizer, num_epochs=10)


Total image paths found: 842
Epoch [1/10], Loss: 0.0000, Accuracy: 100.00%
Validation Accuracy: 100.00%
Model saved!
Epoch [2/10], Loss: 0.0000, Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [3/10], Loss: 0.0000, Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [4/10], Loss: 0.0000, Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [5/10], Loss: 0.0000, Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [6/10], Loss: 0.0000, Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [7/10], Loss: 0.0000, Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [8/10], Loss: 0.0000, Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [9/10], Loss: 0.0000, Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [10/10], Loss: 0.0000, Accuracy: 100.00%
Validation Accuracy: 100.00%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mish Activation Function
class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(nn.functional.softplus(x))

# Custom Dataset
class GrayscaleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("L")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Transforms
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load dataset
dataset_path = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/HEALTHY_GRAY_IMAGES_EXTENDED"
image_paths = []
labels = []

for filename in os.listdir(dataset_path):
    if filename.endswith((".jpg", ".png")):
        image_paths.append(os.path.join(dataset_path, filename))
        labels.append(0)  # Use actual label if multi-class

# Train-test split
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42)

# Datasets and loaders
train_dataset = GrayscaleDataset(train_paths, train_labels, transform=transform_train)
val_dataset = GrayscaleDataset(val_paths, val_labels, transform=transform_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model setup
resnet = models.resnet18(pretrained=True)
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
num_classes = 2  # Use len(set(labels)) for multi-class
resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 128),
    Mish(),
    nn.Dropout(0.5),
    nn.Linear(128, num_classes)
)
resnet.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Train function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {running_loss:.4f} Accuracy: {train_acc:.2f}%")

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_acc = 100 * val_correct / val_total
        #print(f"Validation Accuracy: {val_acc:.2f}%")

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), "best_resnet18_mish_model.pth")
            print("Best model saved!")

        scheduler.step()

# Train
train_model(resnet, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import numpy as np

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mish Activation Function
class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(nn.functional.softplus(x))

# Custom Dataset
class GrayscaleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("L")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Transforms
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load dataset
dataset_path = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/GRAY_IMAGES"
image_paths = []
labels = []

for filename in os.listdir(dataset_path):
    if filename.endswith((".jpg", ".png")):
        image_paths.append(os.path.join(dataset_path, filename))
        labels.append(0)  # Use actual label if multi-class

# Train-test split
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42)

# Datasets and loaders
train_dataset = GrayscaleDataset(train_paths, train_labels, transform=transform_train)
val_dataset = GrayscaleDataset(val_paths, val_labels, transform=transform_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model setup
resnet = models.resnet18(pretrained=True)
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
num_classes = 2  # Use len(set(labels)) for multi-class
resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 128),
    Mish(),
    nn.Dropout(0.5),
    nn.Linear(128, num_classes)
)
resnet.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Directory to save model
model_dir = "/content/drive/MyDrive/CAPSTONE_PROCESSED/models"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, "best_resnet18_mish_model.pth")

# Train function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {running_loss:.4f} Accuracy: {train_acc:.2f}%")

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_acc = 100 * val_correct / val_total
        print(f"Validation Accuracy: {val_acc:.2f}%")

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), model_path)
            print(f"Best model saved at {model_path}")

        scheduler.step()

# Train
train_model(resnet, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10] Loss: 0.9916 Accuracy: 96.73%
Validation Accuracy: 100.00%
Best model saved at /content/drive/MyDrive/CAPSTONE_PROCESSED/models/best_resnet18_mish_model.pth
Epoch [2/10] Loss: 0.0001 Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [3/10] Loss: 0.0000 Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [4/10] Loss: 0.0000 Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [5/10] Loss: 0.0000 Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [6/10] Loss: 0.0001 Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [7/10] Loss: 0.0001 Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [8/10] Loss: 0.0001 Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [9/10] Loss: 0.0002 Accuracy: 100.00%
Validation Accuracy: 100.00%
Epoch [10/10] Loss: 0.0002 Accuracy: 100.00%
Validation Accuracy: 100.00%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define custom Dataset class to load images
class GrayscaleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("L")  # Convert to grayscale
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Set image transformations (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing to match ResNet input size
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalizing grayscale image
])

# Load the dataset paths and labels
dataset_path_down = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/AUTISTIC_GRAY_IMAGES"
dataset_path_healthy = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/NON_AUTISTIC_GRAY_IMAGES"
image_paths = []  # List to hold the image paths
labels = []  # List to hold the labels

# Load Down Syndrome images (label 0)
for filename in os.listdir(dataset_path_down):
    if filename.endswith((".jpg", ".png")):
        image_paths.append(os.path.join(dataset_path_down, filename))
        labels.append(0)  # Label for Down Syndrome

# Load Healthy images (label 1)
for filename in os.listdir(dataset_path_healthy):
    if filename.endswith((".jpg", ".png")):
        image_paths.append(os.path.join(dataset_path_healthy, filename))
        labels.append(1)  # Label for Healthy

# Check if the dataset has been populated
print(f"Total image paths found: {len(image_paths)}")
print(f"Total labels found: {len(labels)}")

# Split the dataset into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = GrayscaleDataset(train_paths, train_labels, transform=transform)
val_dataset = GrayscaleDataset(val_paths, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load a pre-trained ResNet model
resnet = models.resnet18(pretrained=True)

# Modify the first convolution layer to accept single-channel (grayscale) images
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Modify the final fully connected layer to match the number of classes (2 classes: Down Syndrome and Healthy)
resnet.fc = nn.Linear(resnet.fc.in_features, 2)

# Move model to the device (GPU if available)
resnet.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Training loop
num_epochs = 10

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # Backpropagation
            optimizer.step()  # Optimizer step

            # Track training metrics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

        # Validation step
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        print(f"Validation Accuracy: {val_accuracy:.2f}%")

        # Save the best model based on validation accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), "best_resnet_model_autism.pth")
            print("Model saved!")

# Train the model
train_model(resnet, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs)

Total image paths found: 1265
Total labels found: 1265
Epoch [1/10], Loss: 0.8307, Accuracy: 60.97%
Validation Accuracy: 64.43%
Model saved!
Epoch [2/10], Loss: 0.5429, Accuracy: 73.42%
Validation Accuracy: 72.33%
Model saved!
Epoch [3/10], Loss: 0.4871, Accuracy: 77.08%
Validation Accuracy: 63.64%
Epoch [4/10], Loss: 0.4153, Accuracy: 80.43%
Validation Accuracy: 73.91%
Model saved!
Epoch [5/10], Loss: 0.3500, Accuracy: 84.78%
Validation Accuracy: 73.52%
Epoch [6/10], Loss: 0.2411, Accuracy: 89.62%
Validation Accuracy: 75.49%
Model saved!
Epoch [7/10], Loss: 0.2335, Accuracy: 90.42%
Validation Accuracy: 78.26%
Model saved!
Epoch [8/10], Loss: 0.1775, Accuracy: 93.48%
Validation Accuracy: 73.52%
Epoch [9/10], Loss: 0.1934, Accuracy: 91.70%
Validation Accuracy: 68.38%
Epoch [10/10], Loss: 0.1910, Accuracy: 91.70%
Validation Accuracy: 63.64%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define custom Dataset class to load images
class GrayscaleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("L")  # Convert to grayscale
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Set image transformations (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing to match ResNet input size
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalizing grayscale image
])

# Load the dataset paths and labels
dataset_path_down = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/GRAY_IMAGES"
dataset_path_healthy = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/HEALTHY_GRAY_IMAGES"
image_paths = []  # List to hold the image paths
labels = []  # List to hold the labels

# Load Down Syndrome images (label 0)
for filename in os.listdir(dataset_path_down):
    if filename.endswith((".jpg", ".png")):
        image_paths.append(os.path.join(dataset_path_down, filename))
        labels.append(0)  # Label for Down Syndrome

# Load Healthy images (label 1)
for filename in os.listdir(dataset_path_healthy):
    if filename.endswith((".jpg", ".png")):
        image_paths.append(os.path.join(dataset_path_healthy, filename))
        labels.append(1)  # Label for Healthy

# Check if the dataset has been populated
print(f"Total image paths found: {len(image_paths)}")
print(f"Total labels found: {len(labels)}")

# Split the dataset into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = GrayscaleDataset(train_paths, train_labels, transform=transform)
val_dataset = GrayscaleDataset(val_paths, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load a pre-trained ResNet model
resnet = models.resnet18(pretrained=True)

# Modify the first convolution layer to accept single-channel (grayscale) images
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Modify the final fully connected layer to match the number of classes (2 classes: Down Syndrome and Healthy)
resnet.fc = nn.Linear(resnet.fc.in_features, 2)

# Move model to the device (GPU if available)
resnet.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Training loop
num_epochs = 10

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # Backpropagation
            optimizer.step()  # Optimizer step

            # Track training metrics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

        # Validation step
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        print(f"Validation Accuracy: {val_accuracy:.2f}%")

        # Save the best model based on validation accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), "best_resnet_model_DOWN.pth")
            print("Model saved!")

# Train the model
train_model(resnet, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs)

Total image paths found: 1607
Total labels found: 1607


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 114MB/s]


Epoch [1/10], Loss: 0.5314, Accuracy: 77.43%
Validation Accuracy: 80.75%
Model saved!
Epoch [2/10], Loss: 0.2586, Accuracy: 90.27%
Validation Accuracy: 81.99%
Model saved!
Epoch [3/10], Loss: 0.1813, Accuracy: 92.92%
Validation Accuracy: 80.75%
Epoch [4/10], Loss: 0.1191, Accuracy: 95.41%
Validation Accuracy: 92.86%
Model saved!
Epoch [5/10], Loss: 0.1246, Accuracy: 95.33%
Validation Accuracy: 90.37%
Epoch [6/10], Loss: 0.0793, Accuracy: 96.96%
Validation Accuracy: 92.86%
Epoch [7/10], Loss: 0.0936, Accuracy: 97.43%
Validation Accuracy: 87.89%
Epoch [8/10], Loss: 0.1538, Accuracy: 95.18%
Validation Accuracy: 86.65%
Epoch [9/10], Loss: 0.1706, Accuracy: 92.92%
Validation Accuracy: 92.86%
Epoch [10/10], Loss: 0.0363, Accuracy: 98.99%
Validation Accuracy: 90.68%
